In [1]:
from autotemplate.extract_utils import extract_from_rxn_smiles, canon_remap
from autotemplate.run_utils import rdchiralRunText_modified, RemoveReagent, clearIsotope
from autotemplate.graph_utils import mapping_for_gold_multiple_smiles, find_unique_templates_dict, countBCN
import pickle
from tqdm import tqdm
from collections import Counter

import argparse
import os
from tqdm import tqdm
from rdkit import Chem
import matplotlib.pyplot as plt
import pickle

from collections import Counter
from autotemplate.extract_utils import extract_from_rxn_smiles, canon_remap
from autotemplate.run_utils import ReassignMapping, rdchiralRunText_modified, RemoveReagent, clearIsotope
from autotemplate.graph_utils import mapping_for_gold_multiple_smiles, find_unique_templates_dict

import itertools

def canon_remap_mapping_dict_generator(outcome):
    outcome_list = outcome.split(".")
    mapping_dict = dict()
    canon_smiles_list = []
    for i, smiles in enumerate(outcome_list):
        canon_smiles = canon_remap(smiles, return_NumAtom=False, iso = False)
        if mapping_dict.get(canon_smiles):
            mapping_dict.update({canon_smiles: mapping_dict.get(canon_smiles) + [smiles]})
        else:
            mapping_dict.update({canon_smiles: [smiles]})
        canon_smiles_list.append(canon_smiles)
    print(mapping_dict)
    return mapping_dict, '.'.join(canon_smiles_list)

def count_change(rxn_smiles):
    rea, prod = rxn_smiles.split('>>')
    rea_count = rea.count('.') + 1
    prod_count = prod.count('.') + 1
    return '{}>>{}'.format(rea_count, prod_count)

def IsInvalidSmiles(smiles):
    """ Check whether there exists repeated atom mapping number in the SMILES.
        Or the rdkit of molecule is None"""
    mol = Chem.MolFromSmiles(smiles)
    if mol == None:
        return True
    atom_map = []
    for atom in mol.GetAtoms():
        mapnum = atom.GetProp('molAtomMapNumber')
        if mapnum in atom_map:
            print(atom_map + [mapnum])
            return True
        atom_map.append(mapnum)
    else:
        return False

def remap_all_templates(reactants, product, all_templates, retro = True, return_ChiralStereo=True):
    """
    This function is retro template, because we can quickly examine if there is secondary amine functional group
    in product. Some reactions are devoid of reactant, so we need to append it on the reactant site.
    """
    r_gold_smiles = reactants
    r_check, r_numatom = canon_remap(reactants, return_NumAtom=True)
    product, p_numatom = ReassignMapping(product, return_NumAtom=True, iso=return_ChiralStereo)

    # if t == None:
    for template in all_templates:
        outcomes = remap_one_template(template, product)
        if not outcomes: continue

        for outcome in outcomes:
            if IsInvalidSmiles(outcome): continue
            print(outcome)
            mapping_dict, r = canon_remap_mapping_dict_generator(outcome)
            # r = canon_remap(outcome)
            if r == None: continue
            r_copy = r.split('.').copy()
            if not ([i for i in r_check.split('.') if not i in r_copy or r_copy.remove(i)]):
            #if not (False in [s in r.split('.') for s in r_check.split('.')]):
            #all Reaxys reactant records must lie in template-outcome
                if return_ChiralStereo:
                    # add = r.split('.') 1
                    for s in r_check.split('.'): 
                        mapping_dict[s].pop(0) # remove one molecule # 
                        # add.remove(s) 1
                    add = list(itertools.chain(*mapping_dict.values())) #
                    if add: r_gold_smiles = reactants + '.' + '.'.join(add)
                    gold_outcome = mapping_for_gold_multiple_smiles(r_gold_smiles, outcome)
                    return gold_outcome+">>"+product, template
                else:
                    return outcome+">>"+product, template
    return None, None

def remap_one_template(template, target_smiles):
    try:
        outcomes = rdchiralRunText_modified(template, target_smiles)
        return outcomes
    except:
        return

def give_mapnum_for_unmapped_reactant_atoms(rxn_smiles):
    """ 
    Leaving group atoms in reactnat site have no atom mapping number.
    Try to append atom mapping number for those reactant atoms.
    """
    reac, prod = rxn_smiles.split('>>')
    reac = Chem.MolFromSmiles(reac)
    prod = Chem.MolFromSmiles(prod)
    if reac.GetNumAtoms() > prod.GetNumAtoms():
        reac_map = {i+1 for i in range(reac.GetNumAtoms())}
        prod_map = {int(atom.GetProp('molAtomMapNumber')) for atom in prod.GetAtoms()}
        undetermined_map = reac_map.difference(prod_map)
        for atom in reac.GetAtoms():
            if not atom.HasProp('molAtomMapNumber'):
                num = undetermined_map.pop()
                atom.SetAtomMapNum(num)
                if not undetermined_map: break
        return Chem.MolToSmiles(reac) + '>>' + Chem.MolToSmiles(prod)
    else:
        return rxn_smiles

def check_reconstruct(template, rxn_smiles, retro = True):
    """Check whether the template obtained from the function "extract_from_rxn_smiles" is True for the original rxn_smiles. """
    reactants, products = rxn_smiles.split('>>')
    gold_reac = set(canon_remap(reactants).split('.'))
    
    if retro:
        reac_list = remap_one_template(template, products)
        if not reac_list: return False
        for reac in reac_list:
            reac = canon_remap(reac)
            if reac:
                check_reac = set(reac.split('.'))
            else:
                continue
            if gold_reac.issubset(check_reac):
                return True
        return False            
    else:
        pass
    

# temp_data = "data/DielsAlder/processed_data.pkl"
# with open(temp_data, "rb") as f:
#     records = pickle.load(f)
# print(records[0])

In [2]:
f = open("data/DielsAlder/MappingResult_DielsAlder.txt", 'r')
input_file = f.readlines()[12000:16000]
f.close()

""" Remove reagent. 
Remove 
(1) same molecule without shared atom-mapping appear in both reactant and product sites. 
(2) unmapped molecules in both reactant and product sites. """
print('Removing reagent...')
input_file = [(RemoveReagent(clearIsotope(line.split('\t')[0])), line.split('\t')[1].strip('\n')) for line in tqdm(input_file)]

print('Extracting templates...')
templates = []
data = []
for i in tqdm(range(len(input_file))):
    rxn_smiles, reaxys_id = input_file[i]
    template = extract_from_rxn_smiles(rxn_smiles, radius = 0)

    # TODO: avoid using the wrong atom-mapped reaction template
    if check_reconstruct(template, rxn_smiles, retro = True):
        templates.append(template)
    #     data.append({'rxn_smiles': rxn_smiles, 'template': template, 'reaxys id': reaxys_id})
    # else:
    data.append({'rxn_smiles': rxn_smiles, 'template': None, 'reaxys id': reaxys_id})

Removing reagent...


  0%|          | 0/4000 [00:00<?, ?it/s]

100%|██████████| 4000/4000 [00:13<00:00, 305.28it/s]


Extracting templates...


100%|██████████| 4000/4000 [02:39<00:00, 25.08it/s]


In [3]:
templates = Counter(templates)
for key, value in sorted(templates.items(), key = lambda templates : templates[1], reverse=True):
    print(key, value)

[#6:1]1-[#6:2]-[#6:3]-[#6:4]=[#6:5]-[#6:6]-1>>[#6:1]=[#6:2].[#6:3]=[#6:4]-[#6:5]=[#6:6] 89
[#6:3]1-[#6:4]=[#6:5]-[#6:6]-[#6:1]-[#6:2]-1>>[#6:1]=[#6:2].[#6:3]=[#6:4]-[#6:5]=[#6:6] 80
[#6:1]1-[#6:3]-[#6:4]=[#6:5]-[#6:6]-[#6:2]-1>>[#6:1]=[#6:2].[#6:3]=[#6:4]-[#6:5]=[#6:6] 72
[#6:3]1-[#6:1]-[#6:2]-[#6:6]-[#6:5]=[#6:4]-1>>[#6:1]=[#6:2].[#6:3]=[#6:4]-[#6:5]=[#6:6] 64
[#6:5]1=[#6:4]-[#6:3]-[#6:1]-[#6:2]-[#6:6]-1>>[#6:1]=[#6:2].[#6:3]=[#6:4]-[#6:5]=[#6:6] 54
[#6:4]1=[#6:5]-[#6:6]-[#6:2]-[#6:1]-[#6:3]-1>>[#6:1]=[#6:2].[#6:3]=[#6:4]-[#6:5]=[#6:6] 46
[#6:1]-[#6:3]-[#6:4]-[#6:6]-[#6:5]-[#6:2]>>[#6:1]-[#6:2].[#6:3]=[#6:4].[#6:5]=[#6:6] 46
[#6:3]1-[#6:4]=[#6:5]-[#6:6]-[#6:1]=[#6:2]-1>>[#6:1]#[#6:2].[#6:3]=[#6:4]-[#6:5]=[#6:6] 34
[#6:1]-[#6:5]-[#6:6]-[#6:3]-[#6:4]-[#6:2]>>[#6:1]-[#6:2].[#6:3]=[#6:4].[#6:5]=[#6:6] 25
[#6:1]-[#6:4]-[#6:5]-[#6:2]-[#7:3]>>[#6:1].[#6:2]=[#7:3].[#6:4]=[#6:5] 23
[#6:3]1-[#6:4]=[#6:5]-[#6:6]-[#6:1]-[#6:2]-1>>[#6:1]=[#6:2].[#6:3]=[#6:4]/[#6:5]=[#6:6] 20
[#6:1]:[#6:4]:[#6:3]:[

In [4]:
processed_templates, changed_records = find_unique_templates_dict(templates)

Init


In [6]:
changed_records
for prev, current in changed_records.items():
    if countBCN(prev) != countBCN(current):
        print(countBCN(prev), countBCN(current))
        print(prev, current)

8 6
[#6:5]-[#6:6]1-[#6:3]-[#6:1]-[#6:2]-[#6:4]=[#6:7]-1>>[#6:1]=[#6:2].[#6:3]=[#6:4]-[#6:5]=[#6:6]-[#6:7] [#6:1]-[#6:2]1-[#6:6]-[#6:7]-[#6:5]-[#6:4]=[#6:3]-1>>[#6:1]-[#6:2]=[#6:3]-[#6:4]=[#6:5].[#6:6]=[#6:7]
5 4
[#6:2]-[#6:4]-[#6:5]-[#6:3]-[#6:1]>>[#6:1].[#6:2]=[#6:3].[#6:4]=[#6:5] [#6:3]-[#6:2]-[#6:5]-[#6:4]-[#6:1]>>[#6:1].[#6:2]=[#6:3].[#6:4]=[#6:5]
11 8
[#6:9]-[#6:10]1:[#6:11](-[#6:12]):[#6:4](-[#6:5]):[#6:2](-[#6:3]):[#6:1](-[#6:6]):[#6:7]:1-[#6:8]>>O=[#6:1]1-[#6:2](-[#6:3])=[#6:4](-[#6:5])-C(-[#6:6])=[#6:7]-1-[#6:8].[#6:9]-[#6:10]#[#6:11]-[#6:12] [#6:9]-[#6:10]1:[#6:1](-[#6:2]):[#6:3](-[#6:4]):[#6:5](-[#6:6]):[#6:7](-[#6:8]):[#6:11]:1-[#6:12]>>O=C1-[#6:1](-[#6:2])=[#6:3](-[#6:4])-[#6:5](-[#6:6])=[#6:7]-1-[#6:8].[#6:9]-[#6:10]#[#6:11]-[#6:12]
7 6
[#16:1]1-[#6:3]-[#6:4]=[#6:6]-[#6:5]-[#7:2]-1>>[#16:1]=[#7:2].[#6:3]=[#6:4]-[#6:5]=[#6:6] [#16:1]1-[#6:6]-[#6:5]=[#6:4]-[#6:3]-[#7:2]-1>>[#16:1]=[#7:2].[#6:3]=[#6:4]-[#6:5]=[#6:6]
7 6
[#6:7]-[#6:5]1=[#6:4]-[#6:1]-[#6:2]-[#6:3]-[#6:6]-1>>[#

In [7]:
countBCN("[#16:1]1-[#6:3]-[#6:4]=[#6:6]-[#6:5]-[#7:2]-1>>[#16:1]=[#7:2].[#6:3]=[#6:4]-[#6:5]=[#6:6]")
countBCN("[#16:1]1-[#6:6]-[#6:5]=[#6:4]-[#6:3]-[#7:2]-1>>[#16:1]=[#7:2].[#6:3]=[#6:4]-[#6:5]=[#6:6]")

7

In [13]:
import pandas as pd
template_path = "data/Kabachnik-FieldsReaction/all_templates_used.csv"
templates = pd.read_csv(template_path)["template"]
rxn_smiles = "[CH:1]([c:5]1[cH:6][cH:7][n:3][cH:9][cH:10]1)=[O:13]>>[CH3:1][C@@H:2]([NH:3][C@H:4]([c:5]1[cH:6][cH:7][n:8][cH:9][cH:10]1)[P@@:11]([CH3:12])(=[O:13])[c:14]1[cH:15][cH:16][cH:17][cH:18][cH:19]1)[c:20]1[cH:21][cH:22][cH:23][cH:24][cH:25]1"
reac, prod = rxn_smiles.split(">>")

In [14]:
remap_all_templates(reac, prod, templates)

[CH3:1][CH:2]([NH2:3])[c:20]1[cH:21][cH:22][cH:23][cH:24][cH:25]1.[CH:4]([c:5]1[cH:6][cH:7][n:8][cH:9][cH:10]1)=[O:26].[PH:11]([CH3:12])(=[O:13])[c:14]1[cH:15][cH:16][cH:17][cH:18][cH:19]1
{'CC(N)c1ccccc1': ['[CH3:1][CH:2]([NH2:3])[c:20]1[cH:21][cH:22][cH:23][cH:24][cH:25]1'], 'O=Cc1ccncc1': ['[CH:4]([c:5]1[cH:6][cH:7][n:8][cH:9][cH:10]1)=[O:26]'], 'C[PH](=O)c1ccccc1': ['[PH:11]([CH3:12])(=[O:13])[c:14]1[cH:15][cH:16][cH:17][cH:18][cH:19]1']}


('[CH:4]([c:5]1[cH:6][cH:7][n:8][cH:9][cH:10]1)=[O:26].[CH3:1][CH:2]([NH2:3])[c:20]1[cH:21][cH:22][cH:23][cH:24][cH:25]1.[PH:11]([CH3:12])(=[O:13])[c:14]1[cH:15][cH:16][cH:17][cH:18][cH:19]1>>[CH3:1][C@@H:2]([NH:3][C@H:4]([c:5]1[cH:6][cH:7][n:8][cH:9][cH:10]1)[P@@:11]([CH3:12])(=[O:13])[c:14]1[cH:15][cH:16][cH:17][cH:18][cH:19]1)[c:20]1[cH:21][cH:22][cH:23][cH:24][cH:25]1',
 '[#15:2]-[#6:1]-[#7:3]>>O=[#6:1].[#15:2].[#7:3]')

In [15]:
remap_one_template('[#15:2]-[#6:1]-[#7:3]>>O=[#6:1].[#15:2].[#7:3]', prod)

['[CH3:1][CH:2]([NH2:3])[c:20]1[cH:21][cH:22][cH:23][cH:24][cH:25]1.[CH:4]([c:5]1[cH:6][cH:7][n:8][cH:9][cH:10]1)=[O:26].[PH:11]([CH3:12])(=[O:13])[c:14]1[cH:15][cH:16][cH:17][cH:18][cH:19]1']